In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession, types
from pyspark.sql.functions import spark_partition_id, col
from pathlib import Path

In [2]:
spark = SparkSession\
        .builder\
        .appName('Data Skew')\
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/03 18:50:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Set Path
file_path = Path().cwd() / 'Datasets' / 'BankChurners.csv'
file_path = str(file_path)
file_path

'/Users/keerthan/Learnings/Pyspark_Practice/Datasets/BankChurners.csv'

In [4]:
df = spark.read.options(header=True, inferSchema=True).csv(file_path)
df.explain()
df.printSchema()

== Physical Plan ==
FileScan csv [CLIENTNUM#17,Attrition_Flag#18,Customer_Age#19,Gender#20,Dependent_count#21,Education_Level#22,Marital_Status#23,Income_Category#24,Card_Category#25,Months_on_book#26,Total_Relationship_Count#27,Months_Inactive_12_mon#28,Contacts_Count_12_mon#29,Credit_Limit#30,Total_Revolving_Bal#31,Avg_Open_To_Buy#32,Total_Amt_Chng_Q4_Q1#33,Total_Trans_Amt#34,Total_Trans_Ct#35,Total_Ct_Chng_Q4_Q1#36,Avg_Utilization_Ratio#37] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/keerthan/Learnings/Pyspark_Practice/Datasets/BankChurners...., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<CLIENTNUM:int,Attrition_Flag:string,Customer_Age:int,Gender:string,Dependent_count:int,Edu...


root
 |-- CLIENTNUM: integer (nullable = true)
 |-- Attrition_Flag: string (nullable = true)
 |-- Customer_Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Dependent_count: integer (nullable = true)
 |-- Education

In [5]:
df2 = df.repartition(4)
df2.explain()
df2.rdd.getNumPartitions()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Exchange RoundRobinPartitioning(4), REPARTITION_BY_NUM, [plan_id=28]
   +- FileScan csv [CLIENTNUM#17,Attrition_Flag#18,Customer_Age#19,Gender#20,Dependent_count#21,Education_Level#22,Marital_Status#23,Income_Category#24,Card_Category#25,Months_on_book#26,Total_Relationship_Count#27,Months_Inactive_12_mon#28,Contacts_Count_12_mon#29,Credit_Limit#30,Total_Revolving_Bal#31,Avg_Open_To_Buy#32,Total_Amt_Chng_Q4_Q1#33,Total_Trans_Amt#34,Total_Trans_Ct#35,Total_Ct_Chng_Q4_Q1#36,Avg_Utilization_Ratio#37] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/keerthan/Learnings/Pyspark_Practice/Datasets/BankChurners...., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<CLIENTNUM:int,Attrition_Flag:string,Customer_Age:int,Gender:string,Dependent_count:int,Edu...




4

In [6]:
df2.withColumn('partition_count', spark_partition_id()).groupBy('partition_count').count().show()

+---------------+-----+
|partition_count|count|
+---------------+-----+
|              0| 2531|
|              1| 2532|
|              2| 2532|
|              3| 2532|
+---------------+-----+



In [8]:
# Repartitioning using the card category column (Assuming partition column is card_category)
df3 = df.repartition(4, col('Card_Category'))
df3.rdd.getNumPartitions()

4

In [9]:
df3.withColumn('partition_id', spark_partition_id()).groupBy('partition_id').count().show()

+------------+-----+
|partition_id|count|
+------------+-----+
|           0| 9436|
|           2|  691|
+------------+-----+



In [31]:
df3.groupBy('Card_Category').count().show()

+-------------+-----+
|Card_Category|count|
+-------------+-----+
|         Blue| 9436|
|         Gold|  116|
|       Silver|  555|
|     Platinum|   20|
+-------------+-----+

